In [ ]:
from transformers import BertTokenizer

### 1.토크나이저 초기화
- BERT 모델의 입력값을 만들려면 토크나이저부터 선언

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

#### 2. 모델 초기화
- 코드를 수행하면 모델을 초기화할 수 있습니다.
- 여기서 중요한 것은 사용 대상 BERT 모델이 프리트레인할 때 썼던 토크나이저를 그대로 사용해야 벡터 변환에 문제가 없다는 점입니다.
- 모델과 토크나이저의 토큰화 방식이 다를 경우 모델이 엉뚱한 결과를 출력하기 때문이죠.
- 따라서 코드를 실행해 모델을 선언할 때 코드와 동일한 모델 이름을 적용합니다.


In [ ]:
from transformers import BertConfig, BertModel
pretrained_model_config = BertConfig.from_pretrained(
    "beomi/kcbert-base"
)
model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config=pretrained_model_config,
)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

`pretrained_model_config`에는
- BERT 모델을 프리트레인할 때 설정했던 내용이 담겨 있습니다.
- 코랩에서 pretrained_model_config를 입력하면 그림1을 확인할 수 있습니다.
- 블록(레이어) 수는 12개, 헤드의 수는 12개, 어휘 집합의 크기는 3만개 등 정보를 확인할 수 있습니다.

In [ ]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

#### 3. Input값 만들기
- 코드 수행하면 BERT 모델의 입력값을 만들 수 있습니다.
- 두 개의 입력 문장 각각에 대해 워드피스 토큰화를 수행한 뒤 이를 토큰 인덱스로 변환한 결과가 input_ids입니다.
- BERT 모델은 문장 시작에 CLS, 끝에 SEP라는 스페셜 토큰을 추가하기 때문에 문장 두 개 모두 앞뒤에 이들 토큰에 대응하는 인덱스 2, 3이 덧붙여져 있음을 볼 수 있습니다.

- 토큰 최대 길이(max_length)를 10으로 설정하고, 토큰 길이가 이보다 짧으면 최대 길이에 맞게 패딩(0)을 주고(padding="max_length"), 길면 자르는(truncation=True) 것으로 설정해 두었기 때문에 input_ids의 길이는 두 문장 모두 10인걸 확인할 수 있습니다.

In [ ]:
sentences = ["안녕하세요", "하이!"]
features = tokenizer(
    sentences,
    max_length=10,
    padding="max_length",
    truncation=True,
)

In [ ]:
features

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

- attention_mask는 패딩이 아닌 토큰이 1,
- 패딩인 토큰이 0으로 실제 토큰이 자리하는지 아닌지를 정보를 나타냅니다.
- token_type_ids는 세그먼트(segment) 정보로 파인튜닝을 실시할 때는 모두 0을 줍니다.

#### 4. BERT에 태우기
- 파이토치 모델의 입력값 자료형은 파이토치에서 제공하는 텐서(tensor)여야 합니다.
- 따라서 파이썬 리스트(list) 형태의 features를 Tensor로 변환해 줍니다

In [ ]:
# 피처를 토치 텐서로 변환
import torch

features = {k: torch.tensor(v) for k, v in features.items()}

- outputs은 BERT 모델의 여러 출력 결과를 한데 묶은 것입니다.
코랩에서 outputs.last_hidden_state을 확인해 보면 아래 코드와 같은 결과를 볼 수 있습니다.

In [ ]:
outputs = model(**features)
outputs.last_hidden_state

tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
         [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
         [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
         ...,
         [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
         [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
         [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],

        [[ 0.9371, -1.4749,  1.7351,  ..., -0.3426,  0.8050,  0.4031],
         [ 1.6095, -1.7269,  2.7936,  ...,  0.3100, -0.4787, -1.2491],
         [ 0.4861, -0.4569,  0.5712,  ..., -0.1769,  1.1253, -0.2756],
         ...,
         [ 1.2362, -0.6181,  2.0906,  ...,  1.3677,  0.8132, -0.2742],
         [ 0.5409, -0.9652,  1.6237,  ...,  1.2395,  0.9185,  0.1782],
         [ 1.9001, -0.5859,  3.0156,  ...,  1.4967,  0.1924, -0.4448]]],
       grad_fn=<NativeLayerNormBackward0>)

- shape은 [2, 10, 768]입니다. 문장 두 개에 속한 각각의 토큰(최대 길이 10)을 768차원짜리의 벡터로 변환했다는 의미 -> 이들은 입력 단어 각각에 해당하는 BERT의 마지막 레이어 출력 벡터

- 자연어를 벡터로 바꾼 결과를 임베딩(embedding) 또는 리프레젠테이션(representation)이라고 합니다.

- `안녕하세요, 하이!`라는 문장은 그림3에선 단어 수준의 벡터 시퀀스로, 여기에선 문장 수준의 벡터로 변환되었습니다.

- 전자를 단어 수준 임베딩(리프레젠테이션), 후자를 문장 수준 임베딩(리프레젠테이션)이라고 부릅니다.